In [97]:
#!pip install python-Levenshtein

     |████████████████████████████████| 51kB 421kB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144675 sha256=3de8cdf89a9dd90ffc4c82f36e3bccdc781e340fa2605ee873aecd549674cb9c
  Stored in directory: /home/chris/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [138]:
import pandas as pd
import numpy as np

In [139]:
df_transfer = pd.read_csv('../data/player_static.csv')
df_social = pd.read_csv('../data/social_media_followers.csv')
df_dream = pd.read_csv('../data/player_scores_parsed.csv')
df_noten = pd.read_csv('../data/player_noten.csv')


In [140]:
df_transfer_yb = df_transfer[df_transfer['team'] == 'BSC Young Boys']

In [141]:
df_social[['first_name','last_name']] = df_social['player'].str.split(" ", 1, expand=True)
df_transfer_yb['name'] = df_transfer_yb['name'].str.replace('é','e')
df_transfer_yb['name'] = df_transfer_yb['name'].str.replace('-',' ')
df_transfer_yb[['first_name','last_name']] = df_transfer_yb['name'].str.split(" ", 1, expand=True)

In [142]:
df_social_transfer = pd.merge(df_social, df_transfer_yb,  how='left', left_on=['first_name'], right_on = ['first_name'])

In [143]:
df_dream_latest = df_dream[df_dream['season']==2100].groupby(['player_name']).agg({'score':'sum'}).reset_index()

In [144]:
# TODO name mit zweitem vorname

In [145]:
df_merged = pd.merge(df_social_transfer, df_dream_latest,  how='left', left_on=['player'], right_on = ['player_name'])

In [146]:
ordered_columns = ['player', 
                  'team',
                  'transfermarktlink',
                  'picurl',
                  'position', 
                  'instagram',
                  'twitter',
                  'insta_followers',
                  'twitter_followers',
                  'marketvalue',
                  'score']
df_merged_ordered = df_merged[ordered_columns]
df_merged_ordered.columns = ['name', 
                  'team',
                  'url_trans',
                  'url_picture',
                  'position', 
                  'name_instagram',
                  'name_twitter',
                  'follower_instagram',
                  'follower_twitter',
                  'market_value',
                  'fantasy_score']

In [147]:
df_merged_ordered['market_value'].mean()

29160185.185185187

# 1/3 social 1/3 market value 1/3 fantasy score

In [148]:
# note transfer
df_merged_ordered['note'] = 0.0

df_noten['name'] = df_noten['name'].str.lower()

df_merged_ordered['note'] = df_merged_ordered['name'].apply(lambda x: float(df_noten.loc[df_noten['name']==x.lower()]['note'].iloc[0]) if x.lower() in df_noten['name'].values else 0.0)

/home/david/Documents/envs/fuzz/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/david/Documents/envs/fuzz/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [149]:
df_merged_ordered['follower_instagram_norm'] = df_merged_ordered['follower_instagram'].apply(lambda x: (x-df_merged_ordered['follower_instagram'].min())/(df_merged_ordered['follower_instagram'].max()-df_merged_ordered['follower_instagram'].min()))

df_merged_ordered['follower_twitter_norm'] = df_merged_ordered['follower_twitter'].apply(lambda x: (x-df_merged_ordered['follower_twitter'].min())/(df_merged_ordered['follower_twitter'].max()-df_merged_ordered['follower_twitter'].min()))
df_merged_ordered['market_value_norm'] = df_merged_ordered['market_value'].apply(lambda x: (x-df_merged_ordered['market_value'].min())/(df_merged_ordered['market_value'].max()-df_merged_ordered['market_value'].min()))
df_merged_ordered['fantasy_score_norm'] = df_merged_ordered['fantasy_score'].apply(lambda x: (x-df_merged_ordered['fantasy_score'].min())/(df_merged_ordered['fantasy_score'].max()-df_merged_ordered['fantasy_score'].min()))
df_merged_ordered['note_norm'] = df_merged_ordered['note'].apply(lambda x: (x-df_merged_ordered['note'].min())/(df_merged_ordered['note'].max()-df_merged_ordered['note'].min()))

/home/david/Documents/envs/fuzz/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/david/Documents/envs/fuzz/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/david/Documents/envs/fuzz/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is try

In [153]:
df_merged_ordered['fame'] = 0.0
df_merged_ordered['fame'] = df_merged_ordered.apply(lambda x: round(100*(0.25 * df_merged_ordered['follower_instagram_norm']+ 0.25 * df_merged_ordered['follower_twitter_norm'] + 0.5 * df_merged_ordered['note_norm']  ),1),axis=1)

/home/david/Documents/envs/fuzz/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/david/Documents/envs/fuzz/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [70]:
# TODO: Calculate Fame

## Save to CSV

In [154]:
df_merged_ordered.to_csv( '../data/df_static_merged.csv', index=False)

In [155]:
df_merged_ordered

,name,team,url_trans,url_picture,position,name_instagram,name_twitter,follower_instagram,follower_twitter,market_value,fantasy_score,note,follower_instagram_norm,follower_twitter_norm,market_value_norm,fantasy_score_norm,note_norm,fame
0,Marco Wölfli,BSC Young Boys,/marco-wolfli/profil/spieler/4860,https://tmssl.akamaized.net//images/portrait/h...,Goalkeeper,woelfli_1,NaN,11306,0,385000,3,6.0,0.039117,0.000000,0.004003,0.028302,1.000000,51.0
1,David Von Ballmoos,BSC Young Boys,/david-von-ballmoos/profil/spieler/203124,https://tmssl.akamaized.net//images/portrait/h...,Goalkeeper,davidvonballmoos,NaN,10469,0,38500000,0,5.0,0.036221,0.000000,0.466311,0.000000,0.833333,51.0
2,Dario Marzino,BSC Young Boys,/dario-marzino/profil/spieler/254968,https://tmssl.akamaized.net//images/portrait/h...,Goalkeeper,dariomarzino,NaN,1765,0,165000,0,0.0,0.006107,0.000000,0.001334,0.000000,0.000000,51.0
3,Frederik Sörensen,BSC Young Boys,/frederik-sorensen/profil/spieler/91716,https://tmssl.akamaized.net//images/portrait/h...,Centre-Back,frederiksorensen4,NaN,21033,0,16500000,34,5.5,0.072771,0.000000,0.199466,0.320755,0.916667,51.0
4,Mohamed Ali Camara,BSC Young Boys,/mohamed-camara/profil/spieler/426723,https://tmssl.akamaized.net//images/portrait/h...,Centre-Back,camara_mhdali4,NaN,12966,0,33000000,0,0.0,0.044860,0.000000,0.399600,0.000000,0.000000,51.0
5,Cedric Zesiger,BSC Young Boys,/cedric-zesiger/profil/spieler/382478,https://tmssl.akamaized.net//images/portrait/h...,Centre-Back,zesiger5,ZesigerCedric33,4597,149,825000,42,0.0,0.015905,0.003466,0.009340,0.396226,0.000000,51.0
6,Nicolas Bürgy,BSC Young Boys,/nicolas-burgy/profil/spieler/203895,https://tmssl.akamaized.net//images/portrait/h...,Centre-Back,nbuergy,NaN,2906,0,550000,41,3.5,0.010054,0.000000,0.006004,0.386792,0.583333,51.0
7,Nicolas Bürgy,BSC Young Boys,/nicolas-moumi-ngamaleu/profil/spieler/266768,https://tmssl.akamaized.net//images/portrait/h...,RightWinger,nbuergy,NaN,2906,0,66000000,41,3.5,0.010054,0.000000,0.799867,0.386792,0.583333,51.0
8,Ulisses Garcia,BSC Young Boys,/ulisses-garcia/profil/spieler/192616,https://tmssl.akamaized.net//images/portrait/h...,Left-Back,ulissesgarcia20,ulisses_garcia3,15785,1331,19250000,62,4.0,0.054614,0.030961,0.232822,0.584906,0.666667,51.0
9,Saidy Janko,BSC Young Boys,/saidy-janko/profil/spieler/196435,https://tmssl.akamaized.net//images/portrait/h...,Right-Back,saidyjanko22,NaN,59359,0,22000000,0,3.5,0.205372,0.000000,0.266177,0.000000,0.583333,51.0
